In [19]:
import numpy as np
from modules import bayesian
from modules import cubicharmonics
import matplotlib.pyplot as plt
import copy

In [20]:
#Carico i dati in data e li ordino in sdata e grid

data=np.transpose(np.loadtxt('../nlttk.out'))

sdata, grid = bayesianL.generatesorteddata(data, 500)

[1. 0. 0.] [0. 1. 0.] [0. 0. 1.]


In [21]:
#Medio i dati su tutti i punti equivalenti per simmetria cubica e 
#ritorno la lista dei nuovi dati 'simmetrizzati' in gplot e sd
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=bayesianL.datiplot(C, dic, dics)
sd=np.array([dataplot, datasigmaplot])

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


In [22]:
#Calcolo il fit bayesiano con glot e sd come test
mN, SN, y_infer, sy_infer=\
bayesianL.bayesianpol(gplot, sd, 12, 50, 0.001, gplot, ifprint= True, ifwarning= True)

parametri ottimali [  0.63777401 -15.82259565  45.89534171 -76.937357   -60.808212
 111.07693955   1.3633848   37.38946956  20.56046583  44.28996011
 -44.8100864   18.33653288 -25.9387754   56.94422681  -4.01419798
  30.61982881 -48.51083954  -4.70777009 -27.41204261  19.55333907
 -11.54337695   3.31152886   4.19598144  38.90697839  16.91650152
 -11.21136619 -44.87503627 -19.87181085   5.25290477  17.49695593
   8.07417062]
numero di armoniche cubiche 19
valore a kmin inferito 0.5689314911926667 bias 0.6377740143394826 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 1.010088279481692e+22


In [47]:
#Calcolo il best fit su gplot e sd con 30 dati
N=20
mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
bayesianL.bestfit(gplot, sd, N, gplot, ifprintbestfit= False, ifprintfinal= True)

parametri ottimali [  0.8946795  -15.17997518 143.24188856 302.81413237   7.36547693
  65.86650851 -30.0180168   19.97179406  16.02206581  12.78113867
  -0.66722943   3.23254143  -3.61137793]
numero di armoniche cubiche 7
valore a kmin inferito 0.6870054154665128 bias 0.8946795045249266 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 4.089452758369037e+25
grado ottimale 12 grado massimo tentato 28


In [48]:
#Plotto la funzione di evidence che ho massimizzato per il best fit nella cella precedente
%matplotlib widget
plt.plot(log_evidence_vP, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
#Plotto il best fit e i dati (gplot e sd)
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=bayesianL.datiplot(C, dic, dics)
%matplotlib widget

plt.fill_between(np.linalg.norm(gplot[:N], axis=1), y_infer+np.sqrt(sy_infer), y_infer-np.sqrt(sy_infer),  alpha=0.3, color='red')
plt.plot(np.linalg.norm(gplot[:N], axis=1), y_infer, '.-', color='red')
plt.errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot, fmt='.')

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [42]:
#Calcolo il best fit includendo sempre piu' punti 
predb=[]
spredb=[]
Npointsb=[]
for N in range(10, 101):
    mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
    bayesianL.bestfit(gplot, sd, N, gplot, ifprintbestfit=False, ifprintfinal=False)
    Npointsb.append(np.linalg.norm(gplot[N]))
    if (N-10)%(90//10)==0: print(int((N-10)/90*100), '%')
    predb.append(mN[0])
    spredb.append(SN[0,0])

0 %
10 %
20 %
30 %
40 %
50 %
60 %


KeyboardInterrupt: 

In [ ]:
#Calcolo il model averaging su gplot  e sd con 30 punti
N=20
mr, sr, logevidence=\
bayesianL.bayesianmodelprediction(gplot, sd, N, gplot, ifprintmodpred=False, ifprintfinal=True)

In [ ]:
#Calcolo il model averaging includendo sempre piu' punti 
preda=[]
spreda=[]
Npointsa=[]
for N in range(10, 101):
    
    mr, sr, logevidence=\
    bayesianL.bayesianmodelprediction(gplot, sd, N, gplot, ifprintmodpred=False, ifprintfinal=False)
    Npointsa.append(np.linalg.norm(gplot[N]))
    if (N-10)%(90//10)==0: print(int((N-10)/90*100), '%')
    preda.append(mr)
    spreda.append(sr)

In [20]:
%matplotlib widget
fig, ax=plt.subplots(3,constrained_layout=True, figsize=[9,9])


ax[0].set_title('Bayesian model averaging prediction at k=0 with data in range $[: k_{max}]$')

ax[0].errorbar(np.array(Npointsa), np.array(preda), np.sqrt(np.array(spreda)), fmt='.')
ax[0].plot(np.array(Npointsa), np.ones(len(Npointsa))*0.816, label='GK result')
    
ax[0].set_xlabel('$k_{max} (nm^{-1})$')
ax[0].set_ylabel('prediction at k=0')
ax[0].legend()
ax[1].set_title('Bayesian best model prediction at k=0 with data in range $[: k_{max}]$')

ax[1].errorbar(np.array(Npointsb), np.array(predb), np.sqrt(np.array(spredb)), fmt='.')
ax[1].plot(np.array(Npointsb), np.ones(len(Npointsb))*0.816, label='GK result')    
ax[1].set_xlabel('$k_{max} (nm^{-1})$')
ax[1].set_ylabel('prediction at k=0')
ax[1].legend()
ax[2].set_title('Data')

ax[2].errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot, fmt='.', label='3375 water molecules at 300k 30ns') 

ax[2].legend()

ax[2].set_xlabel(r'k ($nm^{-1}$)')
#plt.savefig('nlttcubic.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …